In [16]:
import pandas as pd
import numpy as np

In [17]:
def ChangeDirection(x):
    if x <= 11.25:
        return 'N'
    elif (x > 11.25) & (x <= 33.75):
        return 'NNE'
    elif (x > 33.75) & (x <= 56.25):
        return 'NE'
    elif (x > 56.25) & (x <= 78.75):
        return 'ENE'
    elif (x > 78.75) & (x <= 101.25):
        return 'E'
    elif (x > 101.25) & (x <= 123.75):
        return 'ESE'
    elif (x > 123.75) & (x <= 146.25):
        return 'SE'
    elif (x > 146.25) & (x <= 168.75):
        return 'SSE'
    elif (x > 168.75) & (x <= 191.25):
        return 'S'
    elif (x > 191.25) & (x <= 213.75):
        return 'SSW'
    elif (x > 213.75) & (x <= 236.25):
        return 'SW'
    elif (x > 236.25) & (x <= 258.75):
        return 'WSW'
    elif (x > 258.75) & (x <= 281.25):
        return 'W'
    elif (x > 281.25) & (x <= 303.75):
        return 'WNW'
    elif (x > 303.75) & (x <= 326.25):
        return 'NW'
    elif (x > 326.25) & (x <= 348.75):
        return 'NNW'
    elif (x > 348.75) & (x <= 360):
        return 'N'

# df_train

In [21]:
ulsan_anch_train_final_v2 = pd.read_csv("./v2/ulsan_anch_train_final_v2.csv")
ulsan_drag_train_final_v2 = pd.read_csv("./v2/ulsan_drag_train_final_v2.csv")

khnp_buoy_train_v2 = pd.read_csv("./v2/khnp_buoy_train_v2.csv")
kma_lightbecon_train_v2 = pd.read_csv("./v2/kma_lightbecon_train_v2.csv")
kma_pagobuoy_train_v2 = pd.read_csv("./v2/kma_pagobuoy_train_v2.csv")

### 정박 + 닻끌림 ################################################################################################
# answer 컬럼 추가
ulsan_anch_train_final_v2['answer'] = 0
ulsan_drag_train_final_v2['answer'] = 1

# class 컬럼 추가
ulsan_anch_train_final_v2['class'] = 'anch'
ulsan_drag_train_final_v2['class'] = 'drag'

# merge를 할 때 기준이 되는 데이터 프레임이라 컬럼명 간단하게 변경
ulsan_anch_train_final_v2 = ulsan_anch_train_final_v2.rename(columns = {'ulsan_anch_train_final.num':'num', 'ulsan_anch_train_final.lat':'lat', 'ulsan_anch_train_final.lon':'lon',
                                                                        'ulsan_anch_train_final.sog':'sog', 'ulsan_anch_train_final.cog':'cog', 'ulsan_anch_train_final.hdg':'hdg', 
                                                                        'ulsan_anch_train_final.year':'year', 'ulsan_anch_train_final.month':'month', 'ulsan_anch_train_final.day':'day', 
                                                                        'ulsan_anch_train_final.hour':'hour', 'ulsan_anch_train_final.min':'min', 'ulsan_anch_train_final.sec':'sec'})

ulsan_drag_train_final_v2 = ulsan_drag_train_final_v2.rename(columns = {'ulsan_drag_train_final.num':'num', 'ulsan_drag_train_final.lat':'lat', 'ulsan_drag_train_final.lon':'lon',
                                                                        'ulsan_drag_train_final.sog':'sog', 'ulsan_drag_train_final.cog':'cog', 'ulsan_drag_train_final.hdg':'hdg', 
                                                                        'ulsan_drag_train_final.year':'year', 'ulsan_drag_train_final.month':'month', 'ulsan_drag_train_final.day':'day', 
                                                                        'ulsan_drag_train_final.hour':'hour', 'ulsan_drag_train_final.min':'min', 'ulsan_drag_train_final.sec':'sec'})

# 정박 + 닻끌림
df_anch_drag = pd.concat([ulsan_anch_train_final_v2,ulsan_drag_train_final_v2], join='inner')
df_anch_drag


### 정박 + 닻끌림 + khnp ############################################################################################
df_anch_drag_khnp = pd.merge(left = df_anch_drag, right = khnp_buoy_train_v2, how = 'left',
                             left_on=['year','month','day','hour','min'], right_on = ['khnp_buoy_train.year','khnp_buoy_train.month','khnp_buoy_train.day','khnp_buoy_train.hour','khnp_buoy_train.min'])

df_anch_drag_khnp.drop(columns=['khnp_buoy_train.year','khnp_buoy_train.month','khnp_buoy_train.day','khnp_buoy_train.hour','khnp_buoy_train.min'], inplace = True)


### 정박 + 닻끌림 + khnp + lightbecon ################################################################################
kma_lightbecon_train_v2['kma_lightbecon_train.wd'] = kma_lightbecon_train_v2['kma_lightbecon_train.wd'].map(ChangeDirection) # 16방위로 변경

df_anch_drag_khnp_lightbecon = pd.merge(left = df_anch_drag_khnp, right = kma_lightbecon_train_v2, how = 'left',
                             left_on=['year','month','day','hour','min'], right_on = ['kma_lightbecon_train.year','kma_lightbecon_train.month','kma_lightbecon_train.day','kma_lightbecon_train.hour','kma_lightbecon_train.min'])

df_anch_drag_khnp_lightbecon.drop(columns=['kma_lightbecon_train.year','kma_lightbecon_train.month','kma_lightbecon_train.day','kma_lightbecon_train.hour','kma_lightbecon_train.min'], inplace = True)


### 정박 + 닻끌림 + khnp + lightbecon + pagobuoy #######################################################################
df = kma_pagobuoy_train_v2.loc[:,['kma_pagobuoy_train.max_wh','kma_pagobuoy_train.sig_wh','kma_pagobuoy_train.mean_wh','kma_pagobuoy_train.year','kma_pagobuoy_train.month','kma_pagobuoy_train.day','kma_pagobuoy_train.hour']]

## df_max_wh
# 간절곶 당사 max_wh 평균
df_max_wh = round(df.groupby(['kma_pagobuoy_train.year', 'kma_pagobuoy_train.month', 'kma_pagobuoy_train.day', 'kma_pagobuoy_train.hour']).mean(),2)['kma_pagobuoy_train.max_wh'].to_frame()
df_max_wh = df_max_wh.T.unstack().reset_index()
df_max_wh.drop(columns = ['level_4'], inplace = True)
df_max_wh = df_max_wh.rename(columns = {0:'kma_pagobuoy_train.max_wh'})
df_max_wh

## df_sig_wh
# 간절곶 당사 sig_wh 평균
df_sig_wh = round(df.groupby(['kma_pagobuoy_train.year', 'kma_pagobuoy_train.month', 'kma_pagobuoy_train.day', 'kma_pagobuoy_train.hour']).mean(),2)['kma_pagobuoy_train.sig_wh'].to_frame()
df_sig_wh = df_sig_wh.T.unstack().reset_index()
df_sig_wh.drop(columns = ['level_4'], inplace = True)
df_sig_wh = df_sig_wh.rename(columns = {0:'kma_pagobuoy_train.sig_wh'})
df_sig_wh

## df_mean_wh
# 간절곶 당사 mean_wh 평균
df_mean_wh = round(df.groupby(['kma_pagobuoy_train.year', 'kma_pagobuoy_train.month', 'kma_pagobuoy_train.day', 'kma_pagobuoy_train.hour']).mean(),2)['kma_pagobuoy_train.mean_wh'].to_frame()
df_mean_wh = df_mean_wh.T.unstack().reset_index()
df_mean_wh.drop(columns = ['level_4'], inplace = True)
df_mean_wh = df_mean_wh.rename(columns = {0:'kma_pagobuoy_train.mean_wh'})
df_mean_wh

## df_max_wh + df_sig_wh + df_mean_wh
df_max_sig = pd.merge(df_max_wh, df_sig_wh, how='inner')
df_max_sig_mean = pd.merge(df_max_sig, df_mean_wh, how='inner')
df_max_sig_mean

## kma_pagobuoy_train_v2
kma_pagobuoy_train_v2 = df_max_sig_mean
kma_pagobuoy_train_v2

## df_anch_drag_khnp_lightbecon_pagobuoy
df_anch_drag_khnp_lightbecon_pagobuoy = pd.merge(left = df_anch_drag_khnp_lightbecon, right = kma_pagobuoy_train_v2, how = 'left',
                             left_on=['year','month','day','hour'], right_on = ['kma_pagobuoy_train.year','kma_pagobuoy_train.month','kma_pagobuoy_train.day','kma_pagobuoy_train.hour'])

df_anch_drag_khnp_lightbecon_pagobuoy.drop(columns=['kma_pagobuoy_train.year','kma_pagobuoy_train.month','kma_pagobuoy_train.day','kma_pagobuoy_train.hour'], inplace = True)


### df_train (정박 + 닻끌림 + khnp + lightbecon + pagobuoy) #######################################################################
df_train = df_anch_drag_khnp_lightbecon_pagobuoy

# csv로 저장
df_train.to_csv("./통합데이터/df_train.csv", index=False)

# df_test

In [ ]:
ulsan_anch_drag_test_v2 = pd.read_csv("./v2/ulsan_anch_drag_test_v2.csv")

khnp_buoy_test_v2 = pd.read_csv("./v2/khnp_buoy_test_v2.csv")
kma_lightbecon_test_v2 = pd.read_csv("./v2/kma_lightbecon_test_v2.csv")
kma_pagobuoy_test_v2 = pd.read_csv("./v2/kma_pagobuoy_test_v2.csv")

## 정박 + 닻끌림

# merge를 할 때 기준이 되는 데이터 프레임이라 컬럼명 간단하게 변경
ulsan_anch_drag_test_v2 = ulsan_anch_drag_test_v2.rename(columns = {'ulsan_anch_drag_test.area':'area', 
                                                                    'ulsan_anch_drag_test.num':'num', 'ulsan_anch_drag_test.lat':'lat', 'ulsan_anch_drag_test.lon':'lon',
                                                                    'ulsan_anch_drag_test.sog':'sog', 'ulsan_anch_drag_test.cog':'cog', 'ulsan_anch_drag_test.hdg':'hdg', 
                                                                    'ulsan_anch_drag_test.year':'year', 'ulsan_anch_drag_test.month':'month', 'ulsan_anch_drag_test.day':'day', 
                                                                    'ulsan_anch_drag_test.hour':'hour', 'ulsan_anch_drag_test.min':'min', 'ulsan_anch_drag_test.sec':'sec'})
df_anch_drag = ulsan_anch_drag_test_v2

### 정박 + 닻끌림 + khnp ############################################################################################
df_anch_drag_khnp = pd.merge(left = df_anch_drag, right = khnp_buoy_test_v2, how = 'left',
                             left_on=['year','month','day','hour','min'], right_on = ['khnp_buoy_test.year','khnp_buoy_test.month','khnp_buoy_test.day','khnp_buoy_test.hour','khnp_buoy_test.min'])

df_anch_drag_khnp.drop(columns=['khnp_buoy_test.year','khnp_buoy_test.month','khnp_buoy_test.day','khnp_buoy_test.hour','khnp_buoy_test.min'], inplace = True)

### 정박 + 닻끌림 + khnp + lightbecon ################################################################################
kma_lightbecon_test_v2['kma_lightbecon_test.wd'] = kma_lightbecon_test_v2['kma_lightbecon_test.wd'].map(ChangeDirection)

df_anch_drag_khnp_lightbecon = pd.merge(left = df_anch_drag_khnp, right = kma_lightbecon_test_v2, how = 'left',
                             left_on=['year','month','day','hour','min'], right_on = ['kma_lightbecon_test.year','kma_lightbecon_test.month','kma_lightbecon_test.day','kma_lightbecon_test.hour','kma_lightbecon_test.min'])

df_anch_drag_khnp_lightbecon.drop(columns=['kma_lightbecon_test.year','kma_lightbecon_test.month','kma_lightbecon_test.day','kma_lightbecon_test.hour','kma_lightbecon_test.min'], inplace = True)

### 정박 + 닻끌림 + khnp + lightbecon + pagobuoy #######################################################################
df = kma_pagobuoy_test_v2.loc[:,['kma_pagobuoy_test.max_wh','kma_pagobuoy_test.sig_wh','kma_pagobuoy_test.mean_wh','kma_pagobuoy_test.year','kma_pagobuoy_test.month','kma_pagobuoy_test.day','kma_pagobuoy_test.hour']]

## df_max_wh
# 간절곶 당사 max_wh 평균
df_max_wh = round(df.groupby(['kma_pagobuoy_test.year', 'kma_pagobuoy_test.month', 'kma_pagobuoy_test.day', 'kma_pagobuoy_test.hour']).mean(),2)['kma_pagobuoy_test.max_wh'].to_frame()
df_max_wh = df_max_wh.T.unstack().reset_index()
df_max_wh.drop(columns = ['level_4'], inplace = True)
df_max_wh = df_max_wh.rename(columns = {0:'kma_pagobuoy_test.max_wh'})
df_max_wh

## df_sig_wh
# 간절곶 당사 sig_wh 평균
df_sig_wh = round(df.groupby(['kma_pagobuoy_test.year', 'kma_pagobuoy_test.month', 'kma_pagobuoy_test.day', 'kma_pagobuoy_test.hour']).mean(),2)['kma_pagobuoy_test.sig_wh'].to_frame()
df_sig_wh = df_sig_wh.T.unstack().reset_index()
df_sig_wh.drop(columns = ['level_4'], inplace = True)
df_sig_wh = df_sig_wh.rename(columns = {0:'kma_pagobuoy_test.sig_wh'})
df_sig_wh

## df_mean_wh
# 간절곶 당사 mean_wh 평균
df_mean_wh = round(df.groupby(['kma_pagobuoy_test.year', 'kma_pagobuoy_test.month', 'kma_pagobuoy_test.day', 'kma_pagobuoy_test.hour']).mean(),2)['kma_pagobuoy_test.mean_wh'].to_frame()
df_mean_wh = df_mean_wh.T.unstack().reset_index()
df_mean_wh.drop(columns = ['level_4'], inplace = True)
df_mean_wh = df_mean_wh.rename(columns = {0:'kma_pagobuoy_test.mean_wh'})
df_mean_wh

## df_max_wh + df_sig_wh + df_mean_wh
df_max_sig = pd.merge(df_max_wh, df_sig_wh, how='inner')
df_max_sig_mean = pd.merge(df_max_sig, df_mean_wh, how='inner')
df_max_sig_mean

## kma_pagobuoy_test_v2
kma_pagobuoy_test_v2 = df_max_sig_mean
kma_pagobuoy_test_v2

# ## df_anch_drag_khnp_lightbecon_pagobuoy
df_anch_drag_khnp_lightbecon_pagobuoy = pd.merge(left = df_anch_drag_khnp_lightbecon, right = kma_pagobuoy_test_v2, how = 'left',
                             left_on=['year','month','day','hour'], right_on = ['kma_pagobuoy_test.year','kma_pagobuoy_test.month','kma_pagobuoy_test.day','kma_pagobuoy_test.hour'])

df_anch_drag_khnp_lightbecon_pagobuoy.drop(columns=['kma_pagobuoy_test.year','kma_pagobuoy_test.month','kma_pagobuoy_test.day','kma_pagobuoy_test.hour'], inplace = True)


### df_test (정박 + 닻끌림 + khnp + lightbecon + pagobuoy) #######################################################################
df_test = df_anch_drag_khnp_lightbecon_pagobuoy

# csv로 저장
df_test.to_csv("./통합데이터/df_test.csv", index=False)

#### -> kma_pagobuoy_test 데이터 부족. (날짜에 해당하는 데이터가 원래 없음. 원본에도 없음.)